### DATE를 인덱스로 하여 Upbit, Binance 데이터 합치기

바이낸스 데이터와 업비트 데이터를 가져와 date를 인덱스로 하여 합친 후 결측 데이터를 처리한다.

In [24]:
import pandas as pd

# 바이낸스 데이터 원화로 변경하여 가져오는 함수
def binance_to_KRW(binance, money):
    df_binance_krw = binance[['Open time','Close']].copy()
    money = money.rename(columns={'일자':'date'})
    df_binance_krw['date'] = pd.to_datetime(df_binance_krw['Open time']).dt.strftime('%Y.%m.%d')
    df_binance_krw = df_binance_krw.merge(money[['date', '기준환율']], on='date', how='left')
    df_binance_krw['기준환율'] = df_binance_krw['기준환율'].ffill().bfill()
    df_binance_krw['Close'] = df_binance_krw['Close'].astype('float')
    df_binance_krw['기준환율'] =df_binance_krw['기준환율'].str.replace(',','').astype('float')
    df_binance_krw['Close'] = df_binance_krw['Close'] * df_binance_krw['기준환율']
    df_binance_krw = df_binance_krw[['Open time','Close']]

    return df_binance_krw

binance = pd.read_csv('DATA/Binance_BTC_prices_2023_5min.csv')
money = pd.read_excel('DATA/_기간별평균환율조회(월별)_20240726.xlsx')
df_binance = binance_to_KRW(binance, money)
df_binance

,Open time,Close
0,2023-01-01 00:00:00,2.095529e+07
1,2023-01-01 00:05:00,2.094425e+07
2,2023-01-01 00:10:00,2.093667e+07
3,2023-01-01 00:15:00,2.095473e+07
4,2023-01-01 00:20:00,2.095549e+07
...,...,...
105099,2023-12-31 23:35:00,5.445278e+07
105100,2023-12-31 23:40:00,5.446566e+07
105101,2023-12-31 23:45:00,5.450272e+07
105102,2023-12-31 23:50:00,5.444118e+07


In [25]:
# 업비트 데이터 불러오기
df_upbit = pd.read_csv('DATA/upbit_BTC_prices_2023_5min.csv')

df_upbit = df_upbit[['candle_date_time_utc','trade_price']]
df_upbit = df_upbit.rename(columns={'candle_date_time_utc' : 'Open time',
                             'trade_price'          : 'Close' })
df_upbit['Open time'] = df_upbit['Open time'].str.replace('T', ' ')
df_upbit['Open time']= pd.to_datetime(df_upbit['Open time'])
df_upbit


,Open time,Close
0,2023-01-01 00:00:00,21064000.0
1,2023-01-01 00:05:00,21040000.0
2,2023-01-01 00:10:00,21012000.0
3,2023-01-01 00:15:00,21010000.0
4,2023-01-01 00:20:00,21048000.0
...,...,...
105115,2023-12-31 23:35:00,57070000.0
105116,2023-12-31 23:40:00,57072000.0
105117,2023-12-31 23:45:00,57080000.0
105118,2023-12-31 23:50:00,57018000.0


In [26]:
# 두 개 데이터 합치기
df_binance['Open time'] = pd.to_datetime(df_binance['Open time'])
df_krw = df_upbit.merge(df_binance,on='Open time',how='left')
df_krw = df_krw.rename(columns={'Close_x':'Upbit', 'Close_y':'Binance'})
df_krw

,Open time,Upbit,Binance
0,2023-01-01 00:00:00,21064000.0,2.095529e+07
1,2023-01-01 00:05:00,21040000.0,2.094425e+07
2,2023-01-01 00:10:00,21012000.0,2.093667e+07
3,2023-01-01 00:15:00,21010000.0,2.095473e+07
4,2023-01-01 00:20:00,21048000.0,2.095549e+07
...,...,...,...
105115,2023-12-31 23:35:00,57070000.0,5.445278e+07
105116,2023-12-31 23:40:00,57072000.0,5.446566e+07
105117,2023-12-31 23:45:00,57080000.0,5.450272e+07
105118,2023-12-31 23:50:00,57018000.0,5.444118e+07


In [27]:
# 바이낸스 점검으로 인한 결측
df_krw['Binance'].isna().sum()

16

In [28]:
# binance 결측행을 점검 이전의 값으로 고정하기
df_krw[['Upbit','Binance']].dropna().astype('int')
df_krw['diff'] = df_krw['Upbit'] - df_krw['Binance']
df_krw

,Open time,Upbit,Binance,diff
0,2023-01-01 00:00:00,21064000.0,2.095529e+07,108712.926
1,2023-01-01 00:05:00,21040000.0,2.094425e+07,95751.109
2,2023-01-01 00:10:00,21012000.0,2.093667e+07,75329.563
3,2023-01-01 00:15:00,21010000.0,2.095473e+07,55270.538
4,2023-01-01 00:20:00,21048000.0,2.095549e+07,92510.158
...,...,...,...,...
105115,2023-12-31 23:35:00,57070000.0,5.445278e+07,2617219.660
105116,2023-12-31 23:40:00,57072000.0,5.446566e+07,2606338.554
105117,2023-12-31 23:45:00,57080000.0,5.450272e+07,2577281.198
105118,2023-12-31 23:50:00,57018000.0,5.444118e+07,2576824.260


diff_percentage = (upbit_price / binance_price -1) *100

In [29]:
df_krw['diff_percentage'] = (df_krw['Upbit'] / df_krw['Binance'] - 1) * 100
df_krw

,Open time,Upbit,Binance,diff,diff_percentage
0,2023-01-01 00:00:00,21064000.0,2.095529e+07,108712.926,0.518785
1,2023-01-01 00:05:00,21040000.0,2.094425e+07,95751.109,0.457171
2,2023-01-01 00:10:00,21012000.0,2.093667e+07,75329.563,0.359797
3,2023-01-01 00:15:00,21010000.0,2.095473e+07,55270.538,0.263762
4,2023-01-01 00:20:00,21048000.0,2.095549e+07,92510.158,0.441460
...,...,...,...,...,...
105115,2023-12-31 23:35:00,57070000.0,5.445278e+07,2617219.660,4.806402
105116,2023-12-31 23:40:00,57072000.0,5.446566e+07,2606338.554,4.785288
105117,2023-12-31 23:45:00,57080000.0,5.450272e+07,2577281.198,4.728720
105118,2023-12-31 23:50:00,57018000.0,5.444118e+07,2576824.260,4.733227


In [51]:
# df_krw.to_csv('DATA/btc_diff_percentage_2023.csv')